In [119]:
import pandas_ta as ta
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings


warnings.simplefilter('ignore')

df = pd.read_csv('/Users/leo/Desktop/Invest_hub/data/historical_data/PIKK_BBG004S68BH6.csv')
df

,Time,Open,High,Low,Close,Volume
0,2023-05-22 12:00:00+00:00,689.2,690.7,687.6,689.9,19113
1,2023-05-22 13:00:00+00:00,689.8,689.9,688.1,688.8,2325
2,2023-05-22 14:00:00+00:00,688.8,689.6,688.0,688.7,4695
3,2023-05-22 15:00:00+00:00,689.0,689.3,688.4,688.4,2225
4,2023-05-22 16:00:00+00:00,687.8,689.3,687.8,688.3,2971
...,...,...,...,...,...,...
4789,2024-05-16 16:00:00+00:00,815.8,821.5,813.0,821.5,47684
4790,2024-05-16 17:00:00+00:00,821.4,833.5,820.7,827.3,88070
4791,2024-05-16 18:00:00+00:00,827.5,829.1,825.5,826.6,12843
4792,2024-05-16 19:00:00+00:00,826.6,831.2,826.0,830.9,61637


In [120]:
CustomStrategy = ta.Strategy(
    name="Custom Technical Indicators",
    description="SMA for short, medium, and long term periods, RSI, STOCH, STOCHRSI, MACD, ADX, Williams %R, CCI, ATR, Ultimate Oscillator, ROC, and Bull/Bear Power",
    ta=[
        {"kind": "sma", "length": 5},  # Short-term SMA
        {"kind": "sma", "length": 10},
        {"kind": "sma", "length": 20},
        {"kind": "sma", "length": 50},  # Medium-term SMA
        {"kind": "sma", "length": 100},
        {"kind": "sma", "length": 200},  # Long-term SMA
        {"kind": "rsi", "length": 14},
        {"kind": "stoch", "k": 9, "d": 6},
        {"kind": "stochrsi", "length": 14},
        {"kind": "macd", "fast": 12, "slow": 26},
        {"kind": "adx", "length": 14},
        {"kind": "willr", "length": 14},
        {"kind": "cci", "length": 14},
        {"kind": "atr", "length": 14},
        # Note: Highs/Lows is not a standard indicator in pandas_ta, you might need to define it manually or use another indicator instead
        # {"kind": "highs_lows", "length": 14},
        {"kind": "uo", "fast": 7, "medium": 14, "slow": 28},  # Ultimate Oscillator
        {"kind": "roc", "length": 14},
        {"kind": "psar"}  # Assuming Bull/Bear Power is represented by Parabolic SAR (psar)
    ]
)
df.ta.strategy(CustomStrategy)
df.columns
df

,Time,Open,High,Low,Close,Volume,SMA_5,SMA_10,SMA_20,SMA_50,SMA_100,SMA_200,RSI_14,STOCHk_9_6_3,STOCHd_9_6_3,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ADX_14,DMP_14,DMN_14,WILLR_14,CCI_14_0.015,ATRr_14,UO_7_14_28,ROC_14,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
0,2023-05-22 12:00:00+00:00,689.2,690.7,687.6,689.9,19113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0
1,2023-05-22 13:00:00+00:00,689.8,689.9,688.1,688.8,2325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,687.600000,NaN,0.02,0
2,2023-05-22 14:00:00+00:00,688.8,689.6,688.0,688.7,4695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,687.600000,NaN,0.02,0
3,2023-05-22 15:00:00+00:00,689.0,689.3,688.4,688.4,2225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,687.662000,NaN,0.02,0
4,2023-05-22 16:00:00+00:00,687.8,689.3,687.8,688.3,2971,688.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,687.722760,NaN,0.02,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4789,2024-05-16 16:00:00+00:00,815.8,821.5,813.0,821.5,47684,817.74,824.00,829.780,834.160,838.254,840.1735,37.652308,24.863270,16.562156,25.253080,12.422192,-4.955444,-1.501163,-3.454281,34.344925,16.157413,40.682782,-60.995851,-82.624464,3.662934,42.352744,-1.722694,NaN,829.037043,0.10,0
4790,2024-05-16 17:00:00+00:00,821.4,833.5,820.7,827.3,88070,819.68,823.43,829.395,833.994,838.086,840.0655,47.412876,40.241715,22.606826,55.886285,29.048705,-4.327428,-0.698517,-3.628910,31.950494,32.595938,32.063854,-36.401674,6.077933,4.315582,45.945070,-1.040670,812.100000,NaN,0.02,1
4791,2024-05-16 18:00:00+00:00,827.5,829.1,825.5,826.6,12843,821.30,822.91,828.895,833.818,837.952,839.9710,46.467384,61.253708,31.380069,88.175069,56.438145,-3.841917,-0.170406,-3.671512,29.727094,30.630438,30.130438,-37.768240,11.684470,4.264469,47.206733,-0.958543,812.528000,NaN,0.02,0
4792,2024-05-16 19:00:00+00:00,826.6,831.2,826.0,830.9,61637,823.90,822.78,828.470,833.696,837.835,839.8800,52.706434,75.545171,41.009234,98.955450,81.005602,-3.074729,0.477426,-3.552155,28.078233,31.466904,27.546612,-19.313305,42.419254,4.331292,55.526122,-0.311938,812.947440,NaN,0.02,0


In [121]:
'''
ALGORITHM for using technical indicators to predict whether to buy or sell a stock
'''

def make_decision(df):
    buy_signals = 0
    sell_signals = 0
    neutral_signals = 0
    volatility_signal = "Neutral"

    # RSI
    if df['RSI_14'].iloc[-1] > 70:
        sell_signals += 1
    elif df['RSI_14'].iloc[-1] < 30:
        buy_signals += 1
    else:
        neutral_signals += 1

    # Stochastic Oscillator
    if df['STOCHk_9_6_3'].iloc[-1] > 80:
        sell_signals += 1
    elif df['STOCHk_9_6_3'].iloc[-1] < 20:
        buy_signals += 1
    else:
        neutral_signals += 1

    # Stochastic RSI
    if df['STOCHRSIk_14_14_3_3'].iloc[-1] > 0.8:
        sell_signals += 1
    elif df['STOCHRSIk_14_14_3_3'].iloc[-1] < 0.2:
        buy_signals += 1
    else:
        neutral_signals += 1

    # MACD
    if df['MACDh_12_26_9'].iloc[-1] > 0:  # Histogram positive: buy signal
        buy_signals += 1
    elif df['MACDh_12_26_9'].iloc[-1] < 0:  # Histogram negative: sell signal
        sell_signals += 1
    else:
        neutral_signals += 1

    # ADX
    if df['ADX_14'].iloc[-1] > 25:
        if df['DMP_14'].iloc[-1] > df['DMN_14'].iloc[-1]:
            buy_signals += 1
        elif df['DMP_14'].iloc[-1] < df['DMN_14'].iloc[-1]:
            sell_signals += 1
    else:
        neutral_signals += 1

    # Williams %R
    if df['WILLR_14'].iloc[-1] > -20:
        sell_signals += 1
    elif df['WILLR_14'].iloc[-1] < -80:
        buy_signals += 1
    else:
        neutral_signals += 1

    # CCI
    if df['CCI_14_0.015'].iloc[-1] > 100:
        sell_signals += 1
    elif df['CCI_14_0.015'].iloc[-1] < -100:
        buy_signals += 1
    else:
        neutral_signals += 1

    # Ultimate Oscillator
    if df['UO_7_14_28'].iloc[-1] > 70:
        sell_signals += 1
    elif df['UO_7_14_28'].iloc[-1] < 30:
        buy_signals += 1
    else:
        neutral_signals += 1

    # ROC
    if df['ROC_14'].iloc[-1] > 0:
        buy_signals += 1
    elif df['ROC_14'].iloc[-1] < 0:
        sell_signals += 1
    else:
        neutral_signals += 1
    ''' 
    # PSAR Bull/Bear
    # Assuming that 'PSARl' is Bull and 'PSARs' is Bear, you might consider the trend.
    # You can implement a logic based on the position of PSAR dots in relation to price for bull/bear signals.
    '''

    if df['Close'].iloc[-1] > df['PSARl_0.02_0.2'].iloc[-1]:
        buy_signals += 1
    elif df['Close'].iloc[-1] < df['PSARs_0.02_0.2'].iloc[-1]:
        sell_signals += 1
    else:
        neutral_signals += 1

    current_atr = df['ATRr_14'].iloc[-1]
    historical_atr = df['ATRr_14'].mean()
    atr_threshold_low = 0.5 * historical_atr
    atr_threshold_high = 1.5 * historical_atr
    volatility_signal = "Neutral"
    if current_atr < atr_threshold_low:
        volatility_signal = "Less Volatility"
    elif current_atr > atr_threshold_high:
        volatility_signal = "High Volatility"
    
    decision = {
        'buy_signals': buy_signals,
        'sell_signals': sell_signals,
        'neutral_signals': neutral_signals,
        'volatility': volatility_signal
    }
    return decision

def sma_signals(df, price):
    signals = {}
    sma_periods = [5, 10, 20, 50, 100, 200]

    for period in sma_periods:
        sma_column = f'SMA_{period}'
        if sma_column in df.columns:
            if price > df[sma_column].iloc[-1]:
                signals[sma_column] = 'Buy'
            else:
                signals[sma_column] = 'Sell'

    # Count the Buy and Sell signals
    buy_count = sum(signal == 'Buy' for signal in signals.values())
    sell_count = sum(signal == 'Sell' for signal in signals.values())
    summary_signal = "BUY" if buy_count > sell_count else "SELL" if sell_count > buy_count else "NEUTRAL"

    return {
        "signals": signals,
        "summary": summary_signal,
        "buy_count": buy_count,
        'sell_count': sell_count
    }


current_price = df['Close'].iloc[-1]
sma_decision = sma_signals(df, current_price)
print(f"SMA Signals: {sma_decision['signals']}")

df.ta.strategy(CustomStrategy)

decision = make_decision(df)
print(f"The decision based on the current data is: {decision}")

def overall_decision(df, price):
    indicator_decision = make_decision(df)
    sma_decision = sma_signals(df, price)

    combined_buy_signals = indicator_decision['buy_signals'] + sma_decision['buy_count']
    combined_sell_signals = indicator_decision['sell_signals'] + sma_decision['sell_count']
    combined_neutral_signals = indicator_decision['neutral_signals']  # This line will also need adjustment

    total_combined_signals = combined_buy_signals + combined_sell_signals + combined_neutral_signals
    combined_buy_ratio = combined_buy_signals / total_combined_signals if total_combined_signals else 0
    combined_sell_ratio = combined_sell_signals / total_combined_signals if total_combined_signals else 0
    combined_neutural_ratio = combined_neutral_signals / total_combined_signals if total_combined_signals else 0

    if combined_buy_ratio > 0.7:
        final_decision = "STRONG BUY"
    elif combined_buy_ratio > 0.55:
        final_decision = "BUY"
    elif combined_sell_ratio > 0.7:
        final_decision = "STRONG SELL"
    elif combined_sell_ratio > 0.55:
        final_decision = "SELL"
    elif combined_neutural_ratio > 0.6:
        final_decision = "HOLD"
    elif combined_buy_signals > combined_sell_signals:
        final_decision = "BUY"
    elif combined_sell_signals > combined_buy_signals:
        final_decision = "SELL"
    else:
        final_decision = "HOLD"



    volatility_signal = indicator_decision['volatility']
    return f"{final_decision}, volatility: {volatility_signal}"



final_decision = overall_decision(df, current_price)
print(f'Final decision based on the SMA and indicators is: {final_decision}')





SMA Signals: {'SMA_5': 'Buy', 'SMA_10': 'Buy', 'SMA_20': 'Buy', 'SMA_50': 'Buy', 'SMA_100': 'Sell', 'SMA_200': 'Sell'}


The decision based on the current data is: {'buy_signals': 4, 'sell_signals': 3, 'neutral_signals': 3, 'volatility': 'Neutral'}
Final decision based on the SMA and indicators is: BUY, volatility: Neutral
